# 🏀 LMU Jersey Color Analysis Dashboard - Jan 5th 2026

This notebook loads your pre-built outputs:

- `team_jersey_report.csv` (game-level rollup by jersey color)
- `player_jersey_report2.csv` (player totals + per-game averages by jersey color)

It then displays them with **pretty formatting**, **color gradients**, and a **jersey-color filter**.

# 🏀 LMU Jersey Color Analysis Dashboard

This notebook loads your pre-built outputs:

- `team_jersey_report2.csv` (game-level rollup by jersey color)
- `player_jersey_report3.csv` (player totals + per-game averages by jersey color)

It then displays them with **pretty formatting**, **color gradients**, and a **jersey-color filter**.


In [6]:
import pandas as pd
import numpy as np
from IPython.display import display

TEAM_PATH = r"team_jersey_report2.csv"
PLAYER_PATH = r"player_jersey_report3.csv"

team = pd.read_csv(TEAM_PATH)
players = pd.read_csv(PLAYER_PATH)

# Ensure consistent ordering of jersey colors (optional)
jersey_order = ["white","blue","red","black","gray"]
team["jerseycolor"] = pd.Categorical(team["jerseycolor"], categories=jersey_order, ordered=True)
players["jerseycolor"] = pd.Categorical(players["jerseycolor"], categories=jersey_order, ordered=True)

team = team.sort_values(["jerseycolor"]).reset_index(drop=True)
players = players.sort_values(["jerseycolor","eFG%"], ascending=[True, False]).reset_index(drop=True)

team.head(), players.head()


(  jerseycolor  games  wins  pts_for  pts_against  avg_margin  lmu_efg  \
 0       white      4   3.0    74.75        67.25        3.75    0.467   
 1        blue      2   1.0    73.00        71.00        1.00    0.445   
 2         red      4   1.0    66.25        68.25       -1.00    0.471   
 3       black      3   1.0    60.67        61.33       -0.33    0.399   
 4        gray      1   1.0    63.00        58.00        2.50    0.455   
 
    opp_efg   win%  
 0    0.514  0.750  
 1    0.501  0.500  
 2    0.525  0.250  
 3    0.512  0.333  
 4    0.523  1.000  ,
   jerseycolor            Name  gp  pts  ast  reb  fgm  fga  tpm  pts_mean  \
 0       white  Maya Hernandez   4   64    5   23   27   50    0      16.0   
 1       white     Jess Lawson   4   66    8   31   17   45    1      16.5   
 2       white   Andjela Matic   4   38   17   16   11   35    5       9.5   
 3        blue  Maya Hernandez   2   44    4   20   18   35    0      22.0   
 4        blue     Jess Lawson   2   

In [7]:
# ✅ Quick checks
print("Team rows:", len(team))
print("Player rows:", len(players))

print("\nJersey colors in team report:", team["jerseycolor"].astype(str).unique())
print("Jersey colors in player report:", players["jerseycolor"].astype(str).unique())

# Show any missing jerseycolor rows
missing_team = team[team["jerseycolor"].isna()]
missing_players = players[players["jerseycolor"].isna()]

if len(missing_team):
    display(missing_team)
if len(missing_players):
    display(missing_players.head(20))


Team rows: 5
Player rows: 15

Jersey colors in team report: ['white' 'blue' 'red' 'black' 'gray']
Jersey colors in player report: ['white' 'blue' 'red' 'black' 'gray']


## Team summary

In [8]:
def _fmt_pct(x, decimals=1):
    if pd.isna(x):
        return ""
    return f"{x*100:.{decimals}f}%"

def _fmt_num(x, decimals=1):
    if pd.isna(x):
        return ""
    return f"{x:.{decimals}f}"

def _fmt_int(x):
    if pd.isna(x):
        return ""
    try:
        return f"{int(x)}"
    except Exception:
        return str(x)

def margin_color(val):
    # green for positive, red for negative
    if pd.isna(val):
        return ""
    alpha = min(abs(val) / 15, 1)  # tune denominator for your typical margins
    if val > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if val < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def win_color(val):
    if pd.isna(val):
        return ""
    # val is 0..1
    alpha = min(max(val, 0), 1)
    return f"background-color: rgba(0, 140, 255, {alpha:.2f});"

def efg_color(val):
    if pd.isna(val):
        return ""
    # center around 0.50 (50%)
    diff = val - 0.50
    alpha = min(abs(diff) / 0.10, 1)
    if diff > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if diff < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def opp_efg_color(val):
    if pd.isna(val):
        return ""
    # lower is better, so flip the color logic
    diff = 0.50 - val
    alpha = min(abs(diff) / 0.10, 1)
    if diff > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if diff < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def nice_team_table(df):
    show = df.copy()
    # keep a friendly column order
    cols = ["jerseycolor","games","wins","win%","pts_for","pts_against","avg_margin","lmu_efg","opp_efg"]
    cols = [c for c in cols if c in show.columns]
    show = show[cols]

    sty = (show.style
            .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
            .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
            .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
            .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])
            .format({
                "games": _fmt_int,
                "wins": _fmt_int,
                "pts_for": _fmt_int,
                "pts_against": _fmt_int,
                "avg_margin": lambda x: _fmt_num(x, 2),
                "lmu_efg": lambda x: _fmt_pct(x, 1),
                "opp_efg": lambda x: _fmt_pct(x, 1),
                "win%": lambda x: _fmt_pct(x, 1),
            }, na_rep="")
            .set_properties(**{"font-size":"14px"})
            .set_table_styles([
                {"selector":"th", "props":[("font-size","14px"),("text-align","left")]},
                {"selector":"td", "props":[("text-align","left")]}
            ])
            )
    return sty

def nice_player_table(df):
    show = df.copy()
    # suggested column order
    cols = ["jerseycolor","Name","gp","pts","ast","reb","pts_mean","ast_mean","reb_mean","eFG%","fgm","fga","tpm"]
    cols = [c for c in cols if c in show.columns]
    show = show[cols]

    # Color: eFG% and scoring
    subset_efg = ["eFG%"] if "eFG%" in show.columns else []
    subset_pts = ["pts_mean"] if "pts_mean" in show.columns else []

    sty = (show.style
            .applymap(efg_color, subset=subset_efg)
            .background_gradient(subset=subset_pts, axis=0)  # attempts a nice gradient automatically
            .format({
                "gp": _fmt_int,
                "pts": _fmt_int,
                "ast": _fmt_int,
                "reb": _fmt_int,
                "fgm": _fmt_int,
                "fga": _fmt_int,
                "tpm": _fmt_int,
                "pts_mean": lambda x: _fmt_num(x, 1),
                "ast_mean": lambda x: _fmt_num(x, 1),
                "reb_mean": lambda x: _fmt_num(x, 1),
                "eFG%": lambda x: _fmt_pct(x, 1),
            }, na_rep="")
            .set_properties(**{"font-size":"13px"})
            .set_table_styles([
                {"selector":"th", "props":[("font-size","13px"),("text-align","left")]},
                {"selector":"td", "props":[("text-align","left")]}
            ])
            )
    return sty
nice_team_table(team)


/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:69: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:72: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])


,jerseycolor,games,wins,win%,pts_for,pts_against,avg_margin,lmu_efg,opp_efg
0,white,4,3,75.0%,74,67,3.75,46.7%,51.4%
1,blue,2,1,50.0%,73,71,1.00,44.5%,50.1%
2,red,4,1,25.0%,66,68,-1.00,47.1%,52.5%
3,black,3,1,33.3%,60,61,-0.33,39.9%,51.2%
4,gray,1,1,100.0%,63,58,2.50,45.5%,52.3%


In [9]:
# 📊 Team report by jersey color
display(nice_team_table(team))


/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:69: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_19142/2085106526.py:72: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])


,jerseycolor,games,wins,win%,pts_for,pts_against,avg_margin,lmu_efg,opp_efg
0,white,4,3,75.0%,74,67,3.75,46.7%,51.4%
1,blue,2,1,50.0%,73,71,1.00,44.5%,50.1%
2,red,4,1,25.0%,66,68,-1.00,47.1%,52.5%
3,black,3,1,33.3%,60,61,-0.33,39.9%,51.2%
4,gray,1,1,100.0%,63,58,2.50,45.5%,52.3%


## Player breakdown (filter + sort)

In [5]:
import ipywidgets as widgets

options = ["(all)"] + [c for c in team["jerseycolor"].dropna().astype(str).unique().tolist()]
dropdown = widgets.Dropdown(options=options, value="(all)", description="Jersey:", layout=widgets.Layout(width="250px"))
sort_dropdown = widgets.Dropdown(
    options=["eFG%","pts_mean","ast_mean","reb_mean","gp"],
    value="eFG%",
    description="Sort:",
    layout=widgets.Layout(width="250px")
)
ascending_toggle = widgets.Checkbox(value=False, description="Ascending", indent=False)

display(widgets.HBox([dropdown, sort_dropdown, ascending_toggle]))

def show_filtered(jersey, sort_by, ascending):
    if jersey == "(all)":
        dfp = players.copy()
    else:
        dfp = players[players["jerseycolor"].astype(str) == jersey].copy()
    if sort_by in dfp.columns:
        dfp = dfp.sort_values(sort_by, ascending=ascending)
    display(nice_player_table(dfp.reset_index(drop=True)))

out = widgets.interactive_output(show_filtered, {
    "jersey": dropdown,
    "sort_by": sort_dropdown,
    "ascending": ascending_toggle
})
display(out)


Output()

# 🏀 LMU Jersey Color Analysis Dashboard

This notebook loads your pre-built outputs:

- `team_jersey_report.csv` (game-level rollup by jersey color)
- `player_jersey_report2.csv` (player totals + per-game averages by jersey color)

It then displays them with **pretty formatting**, **color gradients**, and a **jersey-color filter**.


In [4]:
import pandas as pd
import numpy as np
from IPython.display import display

TEAM_PATH = r"team_jersey_report.csv"
PLAYER_PATH = r"player_jersey_report2.csv"

team = pd.read_csv(TEAM_PATH)
players = pd.read_csv(PLAYER_PATH)

# Ensure consistent ordering of jersey colors (optional)
jersey_order = ["white","blue","red","black","gray"]
team["jerseycolor"] = pd.Categorical(team["jerseycolor"], categories=jersey_order, ordered=True)
players["jerseycolor"] = pd.Categorical(players["jerseycolor"], categories=jersey_order, ordered=True)

team = team.sort_values(["jerseycolor"]).reset_index(drop=True)
players = players.sort_values(["jerseycolor","eFG%"], ascending=[True, False]).reset_index(drop=True)

team.head(), players.head()


(  jerseycolor  games  wins  pts_for  pts_against  avg_margin  lmu_efg  \
 0       white      4   3.0    74.75        67.25        3.75    0.467   
 1        blue      1   1.0    64.00        57.00        3.50    0.466   
 2         red      3   1.0    61.67        62.00       -0.17    0.473   
 3       black      2   0.0    57.50        63.00       -2.75    0.375   
 4        gray      1   1.0    63.00        58.00        2.50    0.455   
 
    opp_efg   win%  
 0    0.514  0.750  
 1    0.435  1.000  
 2    0.529  0.333  
 3    0.530  0.000  
 4    0.523  1.000  ,
   jerseycolor            Name  gp  pts  ast  reb  fgm  fga  tpm  pts_mean  \
 0       white  Maya Hernandez   4   64    5   23   27   50    0      16.0   
 1       white     Jess Lawson   4   66    8   31   17   45    1      16.5   
 2       white   Andjela Matic   4   38   17   16   11   35    5       9.5   
 3        blue  Maya Hernandez   1   25    3   10   11   19    0      25.0   
 4        blue     Jess Lawson   1   

In [5]:
# ✅ Quick checks
print("Team rows:", len(team))
print("Player rows:", len(players))

print("\nJersey colors in team report:", team["jerseycolor"].astype(str).unique())
print("Jersey colors in player report:", players["jerseycolor"].astype(str).unique())

# Show any missing jerseycolor rows
missing_team = team[team["jerseycolor"].isna()]
missing_players = players[players["jerseycolor"].isna()]

if len(missing_team):
    display(missing_team)
if len(missing_players):
    display(missing_players.head(20))


Team rows: 5
Player rows: 15

Jersey colors in team report: ['white' 'blue' 'red' 'black' 'gray']
Jersey colors in player report: ['white' 'blue' 'red' 'black' 'gray']


## Team summary

In [12]:
def _fmt_pct(x, decimals=1):
    if pd.isna(x):
        return ""
    return f"{x*100:.{decimals}f}%"

def _fmt_num(x, decimals=1):
    if pd.isna(x):
        return ""
    return f"{x:.{decimals}f}"

def _fmt_int(x):
    if pd.isna(x):
        return ""
    try:
        return f"{int(x)}"
    except Exception:
        return str(x)

def margin_color(val):
    # green for positive, red for negative
    if pd.isna(val):
        return ""
    alpha = min(abs(val) / 15, 1)  # tune denominator for your typical margins
    if val > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if val < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def win_color(val):
    if pd.isna(val):
        return ""
    # val is 0..1
    alpha = min(max(val, 0), 1)
    return f"background-color: rgba(0, 140, 255, {alpha:.2f});"

def efg_color(val):
    if pd.isna(val):
        return ""
    # center around 0.50 (50%)
    diff = val - 0.50
    alpha = min(abs(diff) / 0.10, 1)
    if diff > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if diff < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def opp_efg_color(val):
    if pd.isna(val):
        return ""
    # lower is better, so flip the color logic
    diff = 0.50 - val
    alpha = min(abs(diff) / 0.10, 1)
    if diff > 0:
        return f"background-color: rgba(0, 200, 0, {alpha:.2f});"
    if diff < 0:
        return f"background-color: rgba(255, 0, 0, {alpha:.2f});"
    return ""

def nice_team_table(df):
    show = df.copy()
    # keep a friendly column order
    cols = ["jerseycolor","games","wins","win%","pts_for","pts_against","avg_margin","lmu_efg","opp_efg"]
    cols = [c for c in cols if c in show.columns]
    show = show[cols]

    sty = (show.style
            .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
            .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
            .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
            .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])
            .format({
                "games": _fmt_int,
                "wins": _fmt_int,
                "pts_for": _fmt_int,
                "pts_against": _fmt_int,
                "avg_margin": lambda x: _fmt_num(x, 2),
                "lmu_efg": lambda x: _fmt_pct(x, 1),
                "opp_efg": lambda x: _fmt_pct(x, 1),
                "win%": lambda x: _fmt_pct(x, 1),
            }, na_rep="")
            .set_properties(**{"font-size":"14px"})
            .set_table_styles([
                {"selector":"th", "props":[("font-size","14px"),("text-align","left")]},
                {"selector":"td", "props":[("text-align","left")]}
            ])
            )
    return sty

def nice_player_table(df):
    show = df.copy()
    # suggested column order
    cols = ["jerseycolor","Name","gp","pts","ast","reb","pts_mean","ast_mean","reb_mean","eFG%","fgm","fga","tpm"]
    cols = [c for c in cols if c in show.columns]
    show = show[cols]

    # Color: eFG% and scoring
    subset_efg = ["eFG%"] if "eFG%" in show.columns else []
    subset_pts = ["pts_mean"] if "pts_mean" in show.columns else []

    sty = (show.style
            .applymap(efg_color, subset=subset_efg)
            .background_gradient(subset=subset_pts, axis=0)  # attempts a nice gradient automatically
            .format({
                "gp": _fmt_int,
                "pts": _fmt_int,
                "ast": _fmt_int,
                "reb": _fmt_int,
                "fgm": _fmt_int,
                "fga": _fmt_int,
                "tpm": _fmt_int,
                "pts_mean": lambda x: _fmt_num(x, 1),
                "ast_mean": lambda x: _fmt_num(x, 1),
                "reb_mean": lambda x: _fmt_num(x, 1),
                "eFG%": lambda x: _fmt_pct(x, 1),
            }, na_rep="")
            .set_properties(**{"font-size":"13px"})
            .set_table_styles([
                {"selector":"th", "props":[("font-size","13px"),("text-align","left")]},
                {"selector":"td", "props":[("text-align","left")]}
            ])
            )
    return sty
nice_team_table(team)


/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:69: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:72: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])


,jerseycolor,games,wins,win%,pts_for,pts_against,avg_margin,lmu_efg,opp_efg
0,white,4,3,75.0%,74,67,3.75,46.7%,51.4%
1,blue,1,1,100.0%,64,57,3.50,46.6%,43.5%
2,red,3,1,33.3%,61,62,-0.17,47.3%,52.9%
3,black,2,0,0.0%,57,63,-2.75,37.5%,53.0%
4,gray,1,1,100.0%,63,58,2.50,45.5%,52.3%


In [13]:
# 📊 Team report by jersey color
display(nice_team_table(team))


/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:69: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(margin_color, subset=["avg_margin"] if "avg_margin" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(win_color, subset=["win%" if "win%" in show.columns else []])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(efg_color, subset=["lmu_efg"] if "lmu_efg" in show.columns else [])
/var/folders/qh/zhr4gq8s5px2ff46zwfdk90hy3y30j/T/ipykernel_8324/2085106526.py:72: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(opp_efg_color, subset=["opp_efg"] if "opp_efg" in show.columns else [])


,jerseycolor,games,wins,win%,pts_for,pts_against,avg_margin,lmu_efg,opp_efg
0,white,4,3,75.0%,74,67,3.75,46.7%,51.4%
1,blue,1,1,100.0%,64,57,3.50,46.6%,43.5%
2,red,3,1,33.3%,61,62,-0.17,47.3%,52.9%
3,black,2,0,0.0%,57,63,-2.75,37.5%,53.0%
4,gray,1,1,100.0%,63,58,2.50,45.5%,52.3%


## Player breakdown (filter + sort)

In [8]:
import ipywidgets as widgets

options = ["(all)"] + [c for c in team["jerseycolor"].dropna().astype(str).unique().tolist()]
dropdown = widgets.Dropdown(options=options, value="(all)", description="Jersey:", layout=widgets.Layout(width="250px"))
sort_dropdown = widgets.Dropdown(
    options=["eFG%","pts_mean","ast_mean","reb_mean","gp"],
    value="eFG%",
    description="Sort:",
    layout=widgets.Layout(width="250px")
)
ascending_toggle = widgets.Checkbox(value=False, description="Ascending", indent=False)

display(widgets.HBox([dropdown, sort_dropdown, ascending_toggle]))

def show_filtered(jersey, sort_by, ascending):
    if jersey == "(all)":
        dfp = players.copy()
    else:
        dfp = players[players["jerseycolor"].astype(str) == jersey].copy()
    if sort_by in dfp.columns:
        dfp = dfp.sort_values(sort_by, ascending=ascending)
    display(nice_player_table(dfp.reset_index(drop=True)))

out = widgets.interactive_output(show_filtered, {
    "jersey": dropdown,
    "sort_by": sort_dropdown,
    "ascending": ascending_toggle
})
display(out)


Output()